# **Resumo da Análise Exploratória - Parte 02**
A intenção inicial era utilizar o modelo SVD e similares para gerar um modelo de recomendação de produtos, mas como visto na parte 01 da análise exploratória, não existe realmente um histórico de pedidos e avaliações feitas pelos usuários. Para dar continuidade nesse modelo de recomendação, vamos deixar de lado a recomendação com base no histórico de pedidos e avaliações, e seguiremos com um modelo de recomendação com base em características existentes nas bases de dados.

In [1]:
#Bibliotecas
import pandas as pd

In [2]:
#Datasets
olist_customers = pd.read_csv("olist_customers_dataset.csv", sep=",")
olist_orders = pd.read_csv("olist_orders_dataset.csv", sep=",", usecols=['order_id', 'customer_id', 'order_status'])
olist_order_reviews = pd.read_csv("olist_order_reviews_dataset.csv", sep=",", usecols=['order_id', 'review_score'])
olist_order_items = pd.read_csv("olist_order_items_dataset.csv", sep=",", usecols=['order_id', 'product_id', 'price', 'freight_value'])
olist_products = pd.read_csv("olist_products_dataset.csv", sep=",")

Agora vou fazer alguns filtros que foram feitos na parte 01 para ser reutilizado aqui, como filtrar pedidos que um único tipo de item, com status de entregue e que foram avalidados. Assim amenizando o volume do cruazmento dos dados.

In [3]:
#Lista de produtos únicos nos pedidos
produtos = olist_order_items.groupby('order_id').agg(total=('product_id','count'),
                                          distinto=('product_id','nunique'))
produtos.reset_index(inplace=True)
lista_de_pedidos_com_unico_produto = produtos[produtos['distinto'] == 1]['order_id']

In [4]:
#Filtrando os order_id que estão dentro da lista de pedidos com único tipo de produto
olist_orders = olist_orders[olist_orders['order_id'].isin(lista_de_pedidos_com_unico_produto)]

#Filtrando os order_id que estão dentro da lista de produtos com único tipo de produto
olist_order_items = olist_order_items[olist_order_items['order_id'].isin(lista_de_pedidos_com_unico_produto)]
olist_order_items.drop_duplicates(subset='order_id', inplace=True)

#Filtro de status de pedido como entregue e tirando essa coluna
olist_orders = olist_orders[olist_orders['order_status'] == 'delivered']
olist_orders.drop(columns='order_status', inplace=True)

#Filtrando os pedidos que estão sem avaliações e com order_id duplicados
olist_order_reviews.dropna(subset=['review_score'], inplace=True)
olist_order_reviews.drop_duplicates(subset="order_id", inplace=True)

#Removendo a coluna de Zip Code da base de usuários
olist_customers.drop(columns="customer_zip_code_prefix", inplace=True)

In [5]:
#Cruzamentos
dataset = olist_orders.merge(olist_customers, on="customer_id", how="inner") \
                      .merge(olist_order_items, on="order_id", how="inner") \
                      .merge(olist_products, on="product_id", how="inner") \
                      .merge(olist_order_reviews, on="order_id", how="inner")

In [6]:
#Comparando a qtd de linhas da base nova e da base de pedidos
dataset.shape[0], olist_orders.shape[0], olist_orders.shape[0] - dataset.shape[0]

(92673, 93281, 608)

In [7]:
#Verificando a quantidade de valores vazios
dataset.isnull().sum()

order_id                         0
customer_id                      0
customer_unique_id               0
customer_city                    0
customer_state                   0
product_id                       0
price                            0
freight_value                    0
product_category_name         1317
product_name_lenght           1317
product_description_lenght    1317
product_photos_qty            1317
product_weight_g                16
product_length_cm               16
product_height_cm               16
product_width_cm                16
review_score                     0
dtype: int64

In [ ]:
#Base para exportação
dataset.to_excel('dataset.xlsx', index=False)